# Transient Combustion Modelling

In [2]:
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt

> Author: Elias Aoubala

> Date: 24/10/2025

## 1 - Background

This document contains the authors analysis in the evaluation of transient combustion systems.

## 2 - Practice: Performing Transient Combustion Simulations

In this section, we aim to get some experience on the process of performing chemical kinetic simulations. We will follow the example presented within [this](https://www.youtube.com/watch?v=8y0f2YDHgHk) online video by the maintainers of Cantera.

### 2.1 - Example 1: 0D Perfectly Stirred Reactor

For this example, we simulate a constant volume reaction in Cantera. In effect - we will model a reaction chamber, which has fixed mass flows entering and leaving the chamber - with the volume in the chamber determining the residence time of the gasses. We assume `perfectly stirred conditions` - meaning we dont consider any 2D or 1D effects from mixing of the reactans and it is assumed that everything is perfectly mixed - such that all reactants can contact all other reactants.

In terms of combustion chambers which is our practical use case, this gives us a "ball park" idea of what the combustion conditions inside the chamber will be.

For this reaction, we will be modelling a Hydrogen-Oxygen reaction.

So the first thing we will do is load in our Hydrogen-Oxygen reaction mechanism file. For this, we will use the default `h2_o2.yaml` file contained within the Cantera installation by default.

In [54]:
gas_1 = ct.Solution("h2o2.yaml")

In order to setup this reaction - we need to specify the intial conditions of the solution mixture. To do this, we need to specify the *thermodynamic conditions* (temp/press) along with the composition of the solution at the initial time step. For us, we will specify the following:

- **Temperature** ($T$) [K]
- **Pressure** ($P$) [Pa]
- **Mole Fraction** ($X$) [N.D.]

In rocket propulsion, we don't really work with mole fractions as a practical unit - with mass flows (weighted units) being more commonly used - particular when defining the mixture ratios of the combustion chamber. If one wishes to use these units, we can instead inpose the *mass fractions* contained within the mixture:

- **Mass Fraction** ($Y$) [N.D.]

Whenever we specify the mass fractions or mole fractions - these should always total to 1. If these do not total to 1, cantera will automatically rescale them so they total to 1. For example, if I wanted to do the analysis for a reaction that occurs at an Mixture ration of 4 (4 times the oxidiser as fuel by mass) - I would set the oxidiser mass fraction to 4, and fuel to 1. Cantera would accordingly rescale this so that these numbers equate one.

Following on with the example in the video, we specify the temperature, pressure and mole fraction of the mixture.

In [55]:
gas_1.TPX = 300, ct.one_atm, "H2:1.0, O2:2.0, AR:4.0"

Now we will create two objects for our resevoirs - defining the upstream and downstream composition.

In [56]:
upstream = ct.Reservoir(gas_1)
downstream = ct.Reservoir(gas_1)

Now we can equilibriate our gas mixture.

In [57]:
gas_1.equilibrate("HP")

Finally, we can setup our reactor object so we can do the chemical kinetic reaction.

In [58]:
reactor_1 = ct.IdealGasReactor(gas_1)

Interms of simulating the reactor residence time (time that it sits in the chamber) - we simply just define the mass flow rates as a function of the residence time. The less the mass flow rate for a given volume - the higher the residence time.

In [59]:
residence_time = 1.0e-4

Now we need to define the mass flow controller between the upstream resevoir and our reactor based on this mass flow. Fairly self explanitory.

In [74]:
inlet = ct.MassFlowController(upstream = upstream, 
                              downstream = reactor_1,
                              mdot = reactor_1.mass / residence_time)

For maintaining the pressure in the reactor, we will implmenet a constant pressure controller accordingly.

In [75]:
outlet = ct.PressureController(upstream = reactor_1, 
                               downstream = downstream, 
                               primary=inlet,
                               K=100)

Finally, we can create a reactor network and run the simulation till a steady state is finally achieved. Reactor Networks are in essence an array of reactors in series, which we can connect accordingly. In this case, will only use the one reactor we have setup.

In [76]:
net_1 = ct.ReactorNet([reactor_1])
end_time = 5.0*residence_time 

time = []
T = []
mdot = []

We then enter the for nloop

In [80]:
t = 0
while t < end_time:
    t = net_1.step()
    
    time.append(t)
    T.append(reactor_1.T)

CanteraError: 
*******************************************************************************
CanteraError thrown by PressureController::updateMassFlowRate:
Device is not ready; some parameters have not been set.
*******************************************************************************


In [85]:
import sys

import cantera as ct

gas = ct.Solution('h2o2.yaml')
gas.TPX = 1001.0, ct.one_atm, 'H2:2,O2:1,N2:4'
r = ct.IdealGasConstPressureReactor(gas)

sim = ct.ReactorNet([r])
sim.verbose = True

# limit advance when temperature difference is exceeded
delta_T_max = 20.
r.set_advance_limit('temperature', delta_T_max)

dt_max = 1.e-5
t_end = 100 * dt_max
states = ct.SolutionArray(gas, extra=['t'])

print('{:10s} {:10s} {:10s} {:14s}'.format(
    't [s]', 'T [K]', 'P [Pa]', 'u [J/kg]'))
while sim.time < t_end:
    sim.advance(sim.time + dt_max)
    states.append(r.thermo.state, t=sim.time*1e3)
    print('{:10.3e} {:10.3f} {:10.3f} {:14.6f}'.format(
            sim.time, r.T, r.thermo.P, r.thermo.u))

# Plot the results if matplotlib is installed.
# See http://matplotlib.org/ to get it.
if '--plot' in sys.argv[1:]:
    import matplotlib.pyplot as plt
    plt.clf()
    plt.subplot(2, 2, 1)
    plt.plot(states.t, states.T)
    plt.xlabel('Time (ms)')
    plt.ylabel('Temperature (K)')
    plt.subplot(2, 2, 2)
    plt.plot(states.t, states.X[:, gas.species_index('OH')])
    plt.xlabel('Time (ms)')
    plt.ylabel('OH Mole Fraction')
    plt.subplot(2, 2, 3)
    plt.plot(states.t, states.X[:, gas.species_index('H')])
    plt.xlabel('Time (ms)')
    plt.ylabel('H Mole Fraction')
    plt.subplot(2, 2, 4)
    plt.plot(states.t, states.X[:, gas.species_index('H2')])
    plt.xlabel('Time (ms)')
    plt.ylabel('H2 Mole Fraction')
    plt.tight_layout()
    plt.show()
else:
    print("To view a plot of these results, run this script with the option --plot")

Initializing reactor network.
Reactor 0: 12 variables.
              0 sensitivity params.
Number of equations: 12
Maximum time step:                0
t [s]      T [K]      P [Pa]     u [J/kg]      
 1.000e-05   1001.000 101325.000  620761.940774
 2.000e-05   1001.000 101325.000  620761.940024
 3.000e-05   1001.000 101325.000  620761.937443
 4.000e-05   1001.000 101325.000  620761.932033
 5.000e-05   1001.000 101325.000  620761.922251
 6.000e-05   1001.000 101325.000  620761.905712
 7.000e-05   1001.000 101325.000  620761.878725
 8.000e-05   1001.000 101325.000  620761.835580
 9.000e-05   1001.001 101325.000  620761.767437
 1.000e-04   1001.001 101325.000  620761.660607
 1.100e-04   1001.001 101325.000  620761.493864
 1.200e-04   1001.002 101325.000  620761.234248
 1.300e-04   1001.003 101325.000  620760.830463
 1.400e-04   1001.005 101325.000  620760.202386
 1.500e-04   1001.008 101325.000  620759.224189
 1.600e-04   1001.012 101325.000  620757.696642
 1.700e-04   1001.019 101325.000 

## 2 - C3-v3.5 Complete Mechanism

As a starting point, we will use the complete `c3_v3.5` mechanism available on the Cantera website. This is a huge mechanism - with over 3,761 species and 16,564 reversible reactions. This is not a practical checmical equilibrium file for our use case, and as such we intend to reduce this reactions as much as possible.

In this case, w

In [3]:
k = ct.Solution("c3.yaml")

C:\Users\Elanor\AppData\Local\Temp\ipykernel_17024\599025993.py:1: UserWarning: NasaPoly2::validate: 
For species OHV, discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  53.62056162666667
	Value computed using high-temperature polynomial: 53.5841554314

  k = ct.Solution("c3.yaml")
C:\Users\Elanor\AppData\Local\Temp\ipykernel_17024\599025993.py:1: UserWarning: NasaPoly2::validate: 
For species CHV, discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  107.5046684
	Value computed using high-temperature polynomial: 107.34847808033332

  k = ct.Solution("c3.yaml")
C:\Users\Elanor\AppData\Local\Temp\ipykernel_17024\599025993.py:1: UserWarning: NasaPoly2::validate: 
For species KCHOCH2CO3H, discontinuity in cp/R detected at Tmid = 1000
	Value computed using low-temperature polynomial:  22.934254100000004
	Value computed using high-temperature polynomial: 23.200262099999996

  k = ct.Solution("c3.yaml")
